In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import kagglehub

# General settings
RANDOM_SEED = 42
TEST_SIZE = 0.3

# Load dataset
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# Prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

# Print class distribution
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent: {len(fraud)}, Non-fraudulent: {len(nonfraud)}')
print(f'The positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} '
      f'({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')

# Extract features and labels
X = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y)
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Split data
smote = SMOTE(random_state=RANDOM_SEED)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train.ravel())

rf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=5, class_weight='balanced', random_state=RANDOM_SEED)
rf.fit(X_train_res, y_train_res)

y_probs = rf.predict_proba(X_test_scaled)[:, 1]

best_threshold = 0.663
y_pred = (y_probs >= best_threshold).astype(int)

# Define evaluation function
def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print('         Accuracy:', accuracy)
    print('  Precision Score:', precision)
    print('     Recall Score:', recall)
    print('         F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

evaluation(y_test, y_pred)

Fraudulent: 492, Non-fraudulent: 284315
The positive class (frauds) percentage: 492/284807 (0.173%)

Model Evaluation:
         Accuracy: 0.9989700736163291
  Precision Score: 0.6724137931034483
     Recall Score: 0.7905405405405406
         F1 Score: 0.7267080745341615

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.67      0.79      0.73       148

    accuracy                           1.00     85443
   macro avg       0.84      0.89      0.86     85443
weighted avg       1.00      1.00      1.00     85443

